## This notebook is an example of how to pipeline two models. 
This notebook is an example how to use DeGirum PySDK to do AI inference of a graphical file using two AI models: person detection and pose detection. The person detection model is run on the image and the results are then processed by the pose detection model, one person bbox at a time.
Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with 

0. DeGirum Cloud Server, 
1. AI server equipped with DeGirum ORCA accelerator shared via Peer-to-Peer VPN, 
2. AI server equipped with DeGirum ORCA accelerator running in local network and 
3. AI server equipped with DeGirum ORCA accelerator running on the same machine as this code. 

To try different options, you just need to change the __model_zoo_option_index__ in the code below. 



### Specify your model zoo option here

In [ ]:
model_zoo_option_list = ['DEGIRUM_CLOUD_SERVER', 'P2P_VPN_SERVER', 'LOCAL_NETWORK_SERVER', 'LOCALHOST']
model_zoo_option_index=0

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import cv2 # OpenCV
import os
from dotenv import load_dotenv

envs_loaded=load_dotenv()


In [ ]:
model_zoo_option=model_zoo_option_list[model_zoo_option_index]
if model_zoo_option=='DEGIRUM_CLOUD_SERVER':
    zoo = dg.connect_model_zoo(os.getenv('DEGIRUM_CLOUD_SERVER_ADDRESS'), token=os.getenv('DEGIRUM_CLOUD_TOKEN'))
elif model_zoo_option=='P2P_VPN_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('P2P_VPN_SERVER_ADDRESS'))
elif model_zoo_option=='LOCAL_NETWORK_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_NETWORK_SERVER_ADDRESS'))
elif model_zoo_option=='LOCALHOST':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_HOST_ADDRESS'))
else:
    print('Model zoo option not supported \n')                       

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
people_det_model = zoo.load_model("yolo_v5s_person_det--512x512_quant_n2x_orca_1")
pose_model = zoo.load_model("mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca_1")

# adjust some model properties
pose_model.overlay_show_labels = False
pose_model.overlay_line_width = 1
pose_model.overlay_alpha = 1
pose_model._model_parameters.PoseThreshold = 0.5

In [ ]:
# detect persons
people = people_det_model('./images/ThreePersons.jpg')

# for each detected person detect poses
for person in people.results:
    person_box = people.image.crop(person['bbox'])
    poses = pose_model(person_box)
    display(poses.image_overlay)

In [ ]:
# alternatively, detect all poses
all_poses = pose_model('./images/ThreePersons.jpg')
display(all_poses.image_overlay)